In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import torch
import sys
sys.path.insert(0, "../input/weightedboxesfusion")
from ensemble_boxes import *
import glob

from tqdm import tqdm
import shutil as sh

In [ ]:
!cp -r ../input/yolov5/yolov5/* .

In [ ]:
list(glob.glob("../input/yolov5multiscale/best_*"))

In [ ]:
WEIGHT_FILE1 = '../input/yolov5multiscale/best_yolov5x_fold0.pt'
WEIGHT_FILE2 = '../input/yolov5multiscale/best_yolov5x_fold1.pt'
DIR_TEST = '../input/global-wheat-detection/test/'

## Create Pseudo Labels

In [ ]:
%%time
import argparse

from utils.datasets import *
from utils.utils import *


def detect(save_img=False):
    weights, imgsz = opt.weights,opt.img_size
    source = '../input/global-wheat-detection/test/'
    
    # Initialize
    device = torch_utils.select_device(opt.device)
    half = False
    # Load model
    models = []
    for w in weights:
        models.append(torch.load(w, map_location=device)['model'].to(device).float().eval())


    dataset = LoadImages(source, img_size=1024)

    # Get names and colors

    # Run inference
    t0 = time.time()
    img = torch.zeros((1, 3, imgsz, imgsz), device=device)  # init img
    all_path=[]
    all_bboxex =[]
    all_score =[]
    for path, img, im0s, vid_cap in dataset:
        img = torch.from_numpy(img).to(device)
        img = img.half() if half else img.float()  # uint8 to fp16/32
        img /= 255.0  # 0 - 255 to 0.0 - 1.0
        if img.ndimension() == 3:
            img = img.unsqueeze(0)

        # Inference
        t1 = torch_utils.time_synchronized()
        bboxes_2 = []
        score_2 = []
        for model in models:
            pred = model(img, augment=opt.augment)[0]
            pred = non_max_suppression(pred, 0.4, opt.iou_thres, merge=True, classes=None, agnostic=False)
            t2 = torch_utils.time_synchronized()

            bboxes = []
            score = []
            # Process detections
            for i, det in enumerate(pred):  # detections per image
                p, s, im0 = path, '', im0s
                gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  #  normalization gain whwh
                if det is not None and len(det):
                    det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape).round()
                    for c in det[:, -1].unique():
                        n = (det[:, -1] == c).sum()  # detections per class

                    for *xyxy, conf, cls in det:
                        if True:  # Write to file
                            xywh = torch.tensor(xyxy).view(-1).numpy()  # normalized xywh
                            bboxes.append(xywh)
                            score.append(conf)
            bboxes_2.append(bboxes)
            score_2.append(score)
        all_path.append(path)
        all_score.append(score_2)
        all_bboxex.append(bboxes_2)
    return all_path,all_score,all_bboxex



class opt:
    weights_folds = [WEIGHT_FILE1]
    weights_folds.sort()
    weights = weights_folds
    img_size = 1024
    conf_thres = 0.1
    iou_thres = 0.94
    augment = True
    device = '0'
    classes=None
    agnostic_nms = True
        
opt.img_size = check_img_size(opt.img_size)


with torch.no_grad():
    res = detect()

In [ ]:
def run_wbf(boxes,scores, image_size=1024, iou_thr=0.4, skip_box_thr=0.4, weights=None):
    labels0 = [np.ones(len(scores[idx])) for idx in range(len(scores))]
    boxes, scores, labels = weighted_boxes_fusion(boxes, scores, labels0, weights=None, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    return boxes, scores, labels

In [ ]:
all_path,all_score,all_bboxex = res

In [ ]:
results =[]
testdf_psuedo = []

def format_prediction_string(boxes, scores):
    pred_strings = []
    for j in zip(scores, boxes):
        pred_strings.append("{0:.4f} {1} {2} {3} {4}".format(j[0], j[1][0], j[1][1], j[1][2], j[1][3]))
    return " ".join(pred_strings)


for row in range(len(all_path)):
    image_id = all_path[row].split("/")[-1].split(".")[0]
    boxes = all_bboxex[row]
    scores = all_score[row]
    boxes, scores, labels = run_wbf(boxes,scores)
    boxes = (boxes*1024/1024).astype(np.int32).clip(min=0, max=1023)
    boxes[:, 2] = boxes[:, 2] - boxes[:, 0]
    boxes[:, 3] = boxes[:, 3] - boxes[:, 1]
    
    for box in boxes:
        result = {
            'image_id': 'kaushal28'+image_id,
            'source': 'kaushal28',
            'x': float(box[0]),
            'y': float(box[1]),
            'w': float(box[2]),
            'h': float(box[3]),
            'x_center': float(box[0]) + float(box[2])/2,
            'y_center': float(box[1]) + float(box[3])/2,
            'classes': 0
        }
        testdf_psuedo.append(result)


In [ ]:
test_df_pseudo = pd.DataFrame(testdf_psuedo, columns=['image_id', 'x', 'y', 'w', 'h', 'x_center','y_center','classes', 'source'])
test_df_pseudo.head()

In [ ]:
train_df = pd.read_csv('../input/global-wheat-detection/train.csv')
bboxs = np.stack(train_df['bbox'].apply(lambda x: np.fromstring(x[1:-1], sep=',')))
for i, column in enumerate(['x', 'y', 'w', 'h']):
    train_df[column] = bboxs[:,i]
train_df.drop(columns=['bbox'], inplace=True)
train_df['x_center'] = train_df['x'] + train_df['w']/2
train_df['y_center'] = train_df['y'] + train_df['h']/2
train_df['classes'] = 0

train_df = train_df[['image_id','x', 'y', 'w', 'h','x_center','y_center','classes', 'source']]
train_df.head()

In [ ]:
train_df = pd.concat([train_df, test_df_pseudo], ignore_index=True)
train_df

## Train for a few epochs

In [ ]:
index = list(train_df.image_id.unique())
val_index = index[-50:]
FOLD=0

for image_id, image_meta in tqdm(train_df.groupby('image_id')):
    if image_id in val_index:
        path2save = 'val2017/'
    else:
        path2save = 'train2017/'
    
    if image_meta['source'].unique()[0] == 'kaushal28':
        source = 'test'
        image_id = image_id[9:] # remove kaushal28 from the image_id
    else:
        source = 'train'

    if not os.path.exists('convertor/fold{}/labels/'.format(FOLD)+path2save):
        os.makedirs('convertor/fold{}/labels/'.format(FOLD)+path2save)

    with open('convertor/fold{}/labels/'.format(FOLD) + path2save + image_id + ".txt", 'w+') as f:
        row = image_meta[['classes','x_center','y_center','w','h']].astype(float).values
        row = row/1024
        row = row.astype(str)
        for j in range(len(row)):
            text = ' '.join(row[j])
            f.write(text)
            f.write("\n")

    if not os.path.exists('convertor/fold{}/images/{}'.format(FOLD, path2save)):
        os.makedirs('convertor/fold{}/images/{}'.format(FOLD, path2save))

    sh.copy(f"../input/global-wheat-detection/{source}/{image_id}.jpg", f'convertor/fold{FOLD}/images/{path2save}/{image_id}.jpg')

In [ ]:
data_config = f'''# train and val datasets (image directory or *.txt file with image paths)
train: ./convertor/fold{FOLD}/images/train2017/
val: ./convertor/fold{FOLD}/images/val2017/

# number of classes
nc: 1

# class names
names: ['wheat']'''

with open('wheat_data_config.yaml', 'w') as f:
    f.write(data_config)

In [ ]:
%%writefile yolov5x.yaml

# parameters
nc: 1  # number of classes
depth_multiple: 1.33  # model depth multiple
width_multiple: 1.25  # layer channel multiple

# anchors
anchors:
  - [116,90, 156,198, 373,326]  # P5/32
  - [30,61, 62,45, 59,119]  # P4/16
  - [10,13, 16,30, 33,23]  # P3/8

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]], # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
  ]

# YOLOv5 head
head:
  [[-1, 3, BottleneckCSP, [1024, False]],  # 9

   [-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],
   [-1, 1, nn.Conv2d, [na * (nc + 5), 1, 1]],  # 18 (P3/8-small)

   [-2, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],
   [-1, 1, nn.Conv2d, [na * (nc + 5), 1, 1]],  # 22 (P4/16-medium)

   [-2, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],
   [-1, 1, nn.Conv2d, [na * (nc + 5), 1, 1]],  # 26 (P5/32-large)

   [[], 1, Detect, [nc, anchors]],  # Detect(P5, P4, P3)
  ]

In [ ]:
yolo_config = {
    'epochs': 40,
    'image_size': 1024,
    'batch_size': 2,
    'data_config': 'wheat_data_config.yaml',
    'yolo_config': 'yolov5x.yaml',
    'results_name': f'yolov5x_fold{FOLD}',
    'weights': WEIGHT_FILE1
}

In [ ]:
if len(os.listdir(DIR_TEST)) > 10:
    !(python train.py --img-size {yolo_config.get('image_size')} \
    --batch-size {yolo_config.get('batch_size')} \
    --epochs {yolo_config.get('epochs')} \
    --data {yolo_config.get('data_config')} \
    --cfg {yolo_config.get('yolo_config')} \
    --name {yolo_config.get('results_name')} \
    --weights {yolo_config.get('weights')})

## Train another model

In [ ]:
torch.cuda.empty_cache()
!rm -rf convertor

In [ ]:
class opt:
    weights_folds = [WEIGHT_FILE2]
    weights_folds.sort()
    weights = weights_folds
    img_size = 1024
    conf_thres = 0.1
    iou_thres = 0.94
    augment = True
    device = '0'
    classes=None
    agnostic_nms = True
        
opt.img_size = check_img_size(opt.img_size)


with torch.no_grad():
    res = detect()

In [ ]:
all_path,all_score,all_bboxex = res

In [ ]:
results =[]
testdf_psuedo = []

for row in range(len(all_path)):
    image_id = all_path[row].split("/")[-1].split(".")[0]
    boxes = all_bboxex[row]
    scores = all_score[row]
    boxes, scores, labels = run_wbf(boxes,scores)
    boxes = (boxes*1024/1024).astype(np.int32).clip(min=0, max=1023)
    boxes[:, 2] = boxes[:, 2] - boxes[:, 0]
    boxes[:, 3] = boxes[:, 3] - boxes[:, 1]
    
    for box in boxes:
        result = {
            'image_id': 'kaushal28'+image_id,
            'source': 'kaushal28',
            'x': float(box[0]),
            'y': float(box[1]),
            'w': float(box[2]),
            'h': float(box[3]),
            'x_center': float(box[0]) + float(box[2])/2,
            'y_center': float(box[1]) + float(box[3])/2,
            'classes': 0
        }
        testdf_psuedo.append(result)

In [ ]:
test_df_pseudo = pd.DataFrame(testdf_psuedo, columns=['image_id', 'x', 'y', 'w', 'h', 'x_center','y_center','classes', 'source'])
test_df_pseudo.head()

In [ ]:
train_df = pd.read_csv('../input/global-wheat-detection/train.csv')
bboxs = np.stack(train_df['bbox'].apply(lambda x: np.fromstring(x[1:-1], sep=',')))
for i, column in enumerate(['x', 'y', 'w', 'h']):
    train_df[column] = bboxs[:,i]
train_df.drop(columns=['bbox'], inplace=True)
train_df['x_center'] = train_df['x'] + train_df['w']/2
train_df['y_center'] = train_df['y'] + train_df['h']/2
train_df['classes'] = 0

train_df = train_df[['image_id','x', 'y', 'w', 'h','x_center','y_center','classes', 'source']]
train_df.head()

In [ ]:
train_df = pd.concat([train_df, test_df_pseudo], ignore_index=True)
train_df

In [ ]:
index = list(train_df.image_id.unique())
val_index = index[-50:]
FOLD=1

for image_id, image_meta in tqdm(train_df.groupby('image_id')):
    if image_id in val_index:
        path2save = 'val2017/'
    else:
        path2save = 'train2017/'
    
    if image_meta['source'].unique()[0] == 'kaushal28':
        source = 'test'
        image_id = image_id[9:] # remove kaushal28 from the image_id
    else:
        source = 'train'

    if not os.path.exists('convertor/fold{}/labels/'.format(FOLD)+path2save):
        os.makedirs('convertor/fold{}/labels/'.format(FOLD)+path2save)

    with open('convertor/fold{}/labels/'.format(FOLD) + path2save + image_id + ".txt", 'w+') as f:
        row = image_meta[['classes','x_center','y_center','w','h']].astype(float).values
        row = row/1024
        row = row.astype(str)
        for j in range(len(row)):
            text = ' '.join(row[j])
            f.write(text)
            f.write("\n")

    if not os.path.exists('convertor/fold{}/images/{}'.format(FOLD, path2save)):
        os.makedirs('convertor/fold{}/images/{}'.format(FOLD, path2save))

    sh.copy(f"../input/global-wheat-detection/{source}/{image_id}.jpg", f'convertor/fold{FOLD}/images/{path2save}/{image_id}.jpg')

In [ ]:
torch.cuda.empty_cache()
yolo_config = {
    'epochs': 42,
    'image_size': 1024,
    'batch_size': 2,
    'data_config': 'wheat_data_config.yaml',
    'yolo_config': 'yolov5x.yaml',
    'results_name': f'yolov5x_fold{FOLD}',
    'weights': WEIGHT_FILE2
}

In [ ]:
if len(os.listdir(DIR_TEST)) > 10:
    !(python train.py --img-size {yolo_config.get('image_size')} \
    --batch-size {yolo_config.get('batch_size')} \
    --epochs {yolo_config.get('epochs')} \
    --data {yolo_config.get('data_config')} \
    --cfg {yolo_config.get('yolo_config')} \
    --name {yolo_config.get('results_name')} \
    --weights {yolo_config.get('weights')})

In [ ]:
!rm -rf convertor

In [ ]:
class opt:
    weights_folds = list(glob.glob("weights/best_*")) if len(os.listdir(DIR_TEST)) > 10 else [WEIGHT_FILE1, WEIGHT_FILE2]
    weights_folds.sort()
    weights = weights_folds
    img_size = 1024
    conf_thres = 0.1
    iou_thres = 0.94
    augment = True
    device = '0'
    classes=None
    agnostic_nms = True
        
opt.img_size = check_img_size(opt.img_size)


with torch.no_grad():
    res = detect()

In [ ]:
all_path,all_score,all_bboxex = res

In [ ]:
results =[]
def format_prediction_string(boxes, scores):
    pred_strings = []
    for j in zip(scores, boxes):
        pred_strings.append("{0:.4f} {1} {2} {3} {4}".format(j[0], j[1][0], j[1][1], j[1][2], j[1][3]))
    return " ".join(pred_strings)
for row in range(len(all_path)):
    image_id = all_path[row].split("/")[-1].split(".")[0]
    boxes = all_bboxex[row]
    scores = all_score[row]
    boxes, scores, labels = run_wbf(boxes,scores)
    boxes = (boxes*1024/1024).astype(np.int32).clip(min=0, max=1023)
    boxes[:, 2] = boxes[:, 2] - boxes[:, 0]
    boxes[:, 3] = boxes[:, 3] - boxes[:, 1]
    result = {'image_id': image_id,'PredictionString': format_prediction_string(boxes, scores)}
    results.append(result)
test_df = pd.DataFrame(results, columns=['image_id', 'PredictionString'])

In [ ]:
!rm -rf ./*

In [ ]:
test_df.to_csv('submission.csv', index=False)
test_df.head()

In [ ]:
size = 300
idx =-1
font = cv2.FONT_HERSHEY_SIMPLEX 
image = image = cv2.imread(all_path[idx], cv2.IMREAD_COLOR)
# fontScale 
fontScale = 1
color = (255, 0, 0) 

# Line thickness of 2 px 
thickness = 2
for b,s in zip(boxes,scores):
    image = cv2.rectangle(image, (b[0],b[1]), (b[0]+b[2],b[1]+b[3]), (255,0,0), 1) 
    image = cv2.putText(image, '{:.2}'.format(s), (b[0]+np.random.randint(20),b[1]), font,  
                   fontScale, color, thickness, cv2.LINE_AA)
plt.figure(figsize=[20,20])
plt.imshow(image[:,:,::-1])
plt.show()